In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [14]:
eda_xls = pd.ExcelFile(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndrf-sanction-order_2022-23.xlsx')
xls = pd.ExcelFile(eda_xls)
dfs = []
for sheet in xls.sheet_names:
        print(sheet)
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)
dfs

22-09-2022
22-09-2022_2
22-03-2023
22-03-2023_2
25-03-2023
29-03-2023


[    Sr. No.               Name of Districts  \
 0       1.0                        Bilaspur   
 1       2.0                          Chamba   
 2       3.0                        Hamirpur   
 3       4.0                          Kangra   
 4       5.0                         Kinnaur   
 5       6.0                           Kullu   
 6       7.0                  Lahaul & Spiti   
 7       8.0                           Mandi   
 8       9.0                          Shimla   
 9      10.0                         Sirmour   
 10     11.0                           Solan   
 11     12.0                             Una   
 12      NaN                           Total   
 13      NaN                             NaN   
 14      NaN                             NaN   
 15      NaN                             NaN   
 16      NaN                             NaN   
 17      NaN                             NaN   
 18      NaN                             NaN   
 19      NaN                            

In [28]:
import glob
import os
from pathlib import Path

district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    'Name of District/Department',
    'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]

# your existing function (optionally refactored to return counts)
def load_and_tag(path):
    """
    Read every sheet in the given Excel file, tag with its sheet name as timeperiod,
    and return:
      - dfs:   list of DataFrames
      - found: number of sheets containing any district_columns
      - total: total number of sheets
    """
    xls = pd.ExcelFile(path)
    dfs = []
    sheets_with = 0
    total_sheets = len(xls.sheet_names)

    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)

        if set(df.columns).intersection(district_columns):
            sheets_with += 1

    print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")
    return dfs, sheets_with, total_sheets


def load_all_excels(folder_path):
    """
    Find every .xls/.xlsx in folder_path, run load_and_tag,
    and give you a grand summary at the end.
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_dfs = []
    grand_with = 0
    grand_total = 0

    for path in sorted(excel_paths):
        dfs, sheets_with, total_sheets = load_and_tag(path)
        all_dfs.extend(dfs)
        grand_with += sheets_with
        grand_total += total_sheets

    print()
    print(f"Overall: {grand_with}/{grand_total} sheets (in {len(excel_paths)} files) contain district columns.")
    return all_dfs


# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_dataframes = load_all_excels(folder)


Bank Sanction Order 2023-2024.xlsx: 3/5 sheets have district columns
CB Sanction Order 2023-2024.xlsx: 2/5 sheets have district columns
SDRF-NDRF_2023-24.xlsx: 3/5 sheets have district columns
ndmf-sanction-order_2023-24.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2022-23.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2023-24.xlsx: 8/15 sheets have district columns
sdmf-sanction-order_2021-22.xlsx: 0/3 sheets have district columns
sdmf-sanction-order_2022-23.xlsx: 2/3 sheets have district columns
sdrf-sanction-order_2019-2020.xlsx: 4/7 sheets have district columns
sdrf-sanction-order_2020-21.xlsx: 9/16 sheets have district columns
sdrf-sanction-order_2021-22.xlsx: 3/8 sheets have district columns
sdrf-sanction-order_2022-23.xlsx: 5/10 sheets have district columns
sdrf-sanction-order_2023-24.xlsx: 9/16 sheets have district columns

Overall: 54/105 sheets (in 13 files) contain district columns.


In [29]:
import pandas as pd
import glob
import os
from pathlib import Path

district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    'Name of District/Department',
    'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]

def load_and_tag(path):
    """
    Read every sheet in the given Excel file, tag each with its sheet name as 'timeperiod',
    and return the list of DataFrames.
    """
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)
    return dfs

def load_all_excels_and_report(folder_path):
    """
    Load every .xls/.xlsx in folder_path, collect all sheets,
    and then print:
      - how many sheets per file have district columns
      - overall percentage of rows containing district info
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_dfs = []
    grand_with_sheets = 0
    grand_total_sheets = 0

    for path in sorted(excel_paths):
        # load sheets
        dfs = load_and_tag(path)

        # per-file sheet counts
        sheets_with = 0
        for df in dfs:
            if set(df.columns).intersection(district_columns):
                sheets_with += 1
        total_sheets = len(dfs)
        grand_with_sheets += sheets_with
        grand_total_sheets += total_sheets
        print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")

        all_dfs.extend(dfs)

    # concatenate everything
    big_df = pd.concat(all_dfs, ignore_index=True)

    # identify which of your district_columns actually made it in
    found_cols = list(set(big_df.columns).intersection(district_columns))
    if not found_cols:
        print("No district columns found at all!")
        return all_dfs

    # compute row‐level mask
    mask = big_df[found_cols].notna().any(axis=1)
    total_rows = len(big_df)
    rows_with_info = mask.sum()
    pct_with_info = rows_with_info / total_rows * 100

    print()
    print(f"Overall: {grand_with_sheets}/{grand_total_sheets} sheets contain district columns")
    print(f"Across all {total_rows:,} rows, {rows_with_info:,} ({pct_with_info:.2f}%) have district info")

    return all_dfs

# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_dataframes = load_all_excels_and_report(folder)




Bank Sanction Order 2023-2024.xlsx: 3/5 sheets have district columns
CB Sanction Order 2023-2024.xlsx: 2/5 sheets have district columns
SDRF-NDRF_2023-24.xlsx: 3/5 sheets have district columns
ndmf-sanction-order_2023-24.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2022-23.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2023-24.xlsx: 8/15 sheets have district columns
sdmf-sanction-order_2021-22.xlsx: 0/3 sheets have district columns
sdmf-sanction-order_2022-23.xlsx: 2/3 sheets have district columns
sdrf-sanction-order_2019-2020.xlsx: 4/7 sheets have district columns
sdrf-sanction-order_2020-21.xlsx: 9/16 sheets have district columns
sdrf-sanction-order_2021-22.xlsx: 3/8 sheets have district columns
sdrf-sanction-order_2022-23.xlsx: 5/10 sheets have district columns
sdrf-sanction-order_2023-24.xlsx: 9/16 sheets have district columns

Overall: 54/105 sheets contain district columns
Across all 781 rows, 518 (66.33%) have district info


In [ ]:
import pandas as pd
import glob
import os
import re
from pathlib import Path

# --- your lookup lists ---
district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    #'Name of District/Department',
    #'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]

amount_columns = [
    'Total (Rs. in Crore)', 'Amount (Rs. in Crore)',
    'Total\n(Rs. in crore)', 'Amount (In Rs crore)',
    'Amount (In Crore)', 'Amount (Rs. in crore)',
    'Total (Rs. in crore)', 'Funds released (lakh)'
]

multiplier = {
    'crore': 10000000,
    'crores': 10000000,
    'lakhs': 100000,
    'lakh': 100000,
}


def load_and_tag(path: Path) -> list[pd.DataFrame]:
    """
    Read every sheet in the given Excel file, tag each with its sheet name as 'timeperiod',
    and return the list of DataFrames.
    """
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)
    return dfs


def load_all_excels_and_report(folder_path: str) -> tuple[list[pd.DataFrame], pd.DataFrame]:
    """
    Load every .xls/.xlsx in folder_path, collect all sheets,
    print per-file sheet stats, then:
      - build and return district_df: the concat of only sheets containing district columns
      - also return all_sheets (list of every df) if you need it
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_sheets: list[pd.DataFrame] = []
    district_sheets: list[pd.DataFrame] = []
    grand_with_sheets = grand_total_sheets = 0

    for path in sorted(excel_paths):
        dfs = load_and_tag(Path(path))

        # count how many sheets in this file have any district column
        sheets_with = 0
        for df in dfs:
            if set(df.columns).intersection(district_columns):
                sheets_with += 1
                district_sheets.append(df)
        total_sheets = len(dfs)
        grand_with_sheets += sheets_with
        grand_total_sheets += total_sheets

        print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")
        all_sheets.extend(dfs)

    # build the final district-only dataframe
    if district_sheets:
        district_df = pd.concat(district_sheets, ignore_index=True)
    else:
        print("No district-containing sheets found!")
        district_df = pd.DataFrame()  # empty

    print(f"\nOverall: {grand_with_sheets}/{grand_total_sheets} sheets contain district columns")
    print(f"Combined district‐tables shape: {district_df.shape}")

    return all_sheets, district_df

def extract_amount(s: pd.Series) -> pd.Series:
    """
    Given a string Series like:
       "Rs. 2.00 Crore", "150 Lakh", "1,234/-", "Rs 2.0 (in Lakhs)", "500"
    returns a float Series with the correct multiplier applied, or NaN.
    """

    # Step 1: pull out the first numeric chunk (with commas or dots)
    num_str = (
        s.str.extract(r'([0-9][0-9\.,]*)', expand=False)    # e.g. "2.00", "150", "1,234"
         .str.replace(',', '', regex=False)                # remove thousands commas
    )

    # convert to float; if it fails, becomes NaN
    num = pd.to_numeric(num_str, errors='coerce')

    # Step 2: pull out a unit keyword, if present
    unit_str = (
        s.str.extract(r'\b(lakh|crore)s?\b', flags=re.IGNORECASE, expand=False)
         .str.lower()
    )

    # map unit to multiplier, default=1
    factor = unit_str.map({'lakh': multiplier['lakh'],
                           'crore': multiplier['crore']})
    factor = factor.fillna(1)

    amt = num * factor

    # Step 3: if num was NaN (no digit sequence found), try a plain cleanup
    mask_missing = amt.isna()
    if mask_missing.any():
        fallback = (
            s[mask_missing]
             .str.replace(r'[^\d\.]', '', regex=True)    # keep only digits+dot
        )
        amt.loc[mask_missing] = pd.to_numeric(fallback, errors='coerce')

    return amt

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) Merge all district_columns into one 'district' column
    dist_cols = [c for c in district_columns if c in df.columns]
    if dist_cols:
        df['district'] = df[dist_cols].bfill(axis=1).iloc[:, 0]
        df.drop(columns=dist_cols, inplace=True)

    # 2) Build a single sanction_amount by sweeping through every amt_col
    sanction = pd.Series(index=df.index, dtype='float64')
    for col in amount_columns:
        if col in df.columns:
            series = df[col].astype(str).str.strip()
            amt = extract_amount(series)
            # fill only where still missing
            sanction = sanction.fillna(amt)

    df['sanction_amount'] = sanction
    return df

    # (optional) drop the original amt_cols if you like:
    # df.drop(columns=[c for c in amount_columns if c in df.columns], inplace=True)

'''
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    - Rename any of the district_columns to 'district'
    - Find the first amount column present, strip commas or '/-', convert to float
      then multiply by the correct factor and store in 'sanction_amount'
    """
    df = df.copy()

    # 1) Rename district columns
    rename_map = {col: 'district' for col in district_columns if col in df.columns}
    if rename_map:
        df.rename(columns=rename_map, inplace=True)

    # 2) Handle amount columns
    #    We look for whichever amount-column is in df.columns
    for amt_col in amount_columns:
        if amt_col in df.columns:
            # strip commas and '/-'
            df[amt_col] = (
                df[amt_col]
                .astype(str)
                .str.replace(r'[,\-/]', '', regex=True)
                .replace('', '0')  # in case blank strings appear
                .astype(float)
            )
            # pick multiplier based on the word in column name
            unit = 'lakh' if 'lakh' in amt_col.lower() else 'crore'
            df['sanction_amount'] = df[amt_col] * multiplier[unit]
            break  # only use the first matching amount column

    return df
'''

# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_sheets, district_df = load_all_excels_and_report(folder)

# Now clean only the rows/tables that had district info
cleaned_df = clean_data(district_df)

print("\nCleaned district‐tables shape:", cleaned_df.shape)
cleaned_df.head()


<>:153: SyntaxWarning: invalid escape sequence '\-'
<>:153: SyntaxWarning: invalid escape sequence '\-'
C:\Users\saura\AppData\Local\Temp\ipykernel_53988\2096545721.py:153: SyntaxWarning: invalid escape sequence '\-'
  '''


Bank Sanction Order 2023-2024.xlsx: 2/5 sheets have district columns
CB Sanction Order 2023-2024.xlsx: 0/5 sheets have district columns
SDRF-NDRF_2023-24.xlsx: 2/5 sheets have district columns
ndmf-sanction-order_2023-24.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2022-23.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2023-24.xlsx: 8/15 sheets have district columns
sdmf-sanction-order_2021-22.xlsx: 0/3 sheets have district columns
sdmf-sanction-order_2022-23.xlsx: 2/3 sheets have district columns
sdrf-sanction-order_2019-2020.xlsx: 2/7 sheets have district columns
sdrf-sanction-order_2020-21.xlsx: 9/16 sheets have district columns
sdrf-sanction-order_2021-22.xlsx: 3/8 sheets have district columns
sdrf-sanction-order_2022-23.xlsx: 5/10 sheets have district columns
sdrf-sanction-order_2023-24.xlsx: 9/16 sheets have district columns

Overall: 48/105 sheets contain district columns
Combined district‐tables shape: (561, 150)

Cleaned district‐tables shape: (561, 1

,Sr. No.,2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges),2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance),2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.),Total,timeperiod,Account Head,Amount,2245-02-113-01-SOON-NP(OC) Repair/Construction of Houses Assistance,2245-02-193-01-SOON-NP(OC) Assistance to Local Bodies and other Non-Govt Bodies/Inst.,...,2245-02-106-01-S00N-NP(OC) Repairs & Restoration of Damaged roads & Bridges,"2245-02-109-01-S00N-NP(OC) R & R of Damaged Water Supply, Drainage & sewerage works",2245-02-193-01-S00N-NP(OC) Assistance to Local Bodies & other Non-Govt Bodies/Inst.,2245-02-111-01-S00N-NP(OC) Ex-Gratia Payments to Bereaved Families,2245-02-111-02-S00N-NP(OC) Ex-Gratia-II Death Due to State Specific Disaster,2245-02-111-02-S00N-NP(OC) Death Due to State Specific Disaster,2245-02-101-01-S00N-NP(OC) Cash Dole (Immediate Relief),2245-02-106-01-S00N-NP(OC) Repair & Restoration of Damaged Road & Bridges,district,sanction_amount
0,1,"5,00,00,000/-",-,-,"5,00,00,000/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chamba,NaN
1,2,"2,00,00,000/-","1,00,00,000/-","1,00,00,000/-","4,00,00,000/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hamirpur,NaN
2,3,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kangra,NaN
3,4,-,"3,00,00,000/-","2,02,25,400/-","5,02,25,400/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kullu,NaN
4,5,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mandi,NaN


In [44]:
district_df.columns.to_list()

['Sr. No.',
 'Name of Distt.',
 '2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges)',
 '2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance)',
 '2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.)',
 'Total',
 'timeperiod',
 'Name of District',
 'Account Head',
 'Amount',
 '2245-02-113-01-SOON-NP(OC) Repair/Construction of Houses Assistance',
 '2245-02-193-01-SOON-NP(OC) Assistance to Local Bodies and other Non-Govt Bodies/Inst.',
 'Name of Building',
 'Head of Account',
 'Total Amount Approved',
 'Amount of 1st Instalment (@30%)',
 'Total Amount Approved (₹)',
 'Already Released Amount (₹)',
 'Amount to be Released (₹)',
 'Unnamed: 7',
 'Unnamed: 8',
 'Unnamed: 9',
 'SN',
 'Hazard',
 'District',
 'Title of the Proposal',
 'Implementing Agencies',
 'Total Estimated Amount (In Rs.)',
 '1st Instalment Amount to be Released (₹) @29%',
 'Name of Districts',
 '05-2245-02-101-01-C00N-NP-(OC) Cash Dole (Immediate re

In [42]:
for col in amount_columns:
    print (district_df[col].dtype)

float64
float64
float64


KeyError: 'Amount (In Rs crore)'

In [38]:
cleaned_df

,Sr. No.,2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges),2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance),2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.),Total,timeperiod,Account Head,Amount,2245-02-113-01-SOON-NP(OC) Repair/Construction of Houses Assistance,2245-02-193-01-SOON-NP(OC) Assistance to Local Bodies and other Non-Govt Bodies/Inst.,...,2245-02-106-01-S00N-NP(OC) Repairs & Restoration of Damaged roads & Bridges,"2245-02-109-01-S00N-NP(OC) R & R of Damaged Water Supply, Drainage & sewerage works",2245-02-193-01-S00N-NP(OC) Assistance to Local Bodies & other Non-Govt Bodies/Inst.,2245-02-111-01-S00N-NP(OC) Ex-Gratia Payments to Bereaved Families,2245-02-111-02-S00N-NP(OC) Ex-Gratia-II Death Due to State Specific Disaster,2245-02-111-02-S00N-NP(OC) Death Due to State Specific Disaster,2245-02-101-01-S00N-NP(OC) Cash Dole (Immediate Relief),2245-02-106-01-S00N-NP(OC) Repair & Restoration of Damaged Road & Bridges,district,sanction_amount
0,1,"5,00,00,000/-",-,-,"5,00,00,000/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chamba,NaN
1,2,"2,00,00,000/-","1,00,00,000/-","1,00,00,000/-","4,00,00,000/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hamirpur,NaN
2,3,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kangra,NaN
3,4,-,"3,00,00,000/-","2,02,25,400/-","5,02,25,400/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kullu,NaN
4,5,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mandi,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,11,NaN,NaN,NaN,"35,00,000/-",01-03-2024,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,–,Solan,NaN
557,12,NaN,NaN,NaN,"39,91,000/-",01-03-2024,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,–,Una,NaN
558,Grand Total,NaN,NaN,NaN,NaN,01-03-2024,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"6,49,91,000/-",–,NaN
559,1,NaN,NaN,NaN,"50,00,000/-",12-03-2024,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"50,00,000/-",NaN,NaN,NaN,Chamba,NaN


In [ ]:
import pandas as pd
import glob
import os
from pathlib import Path

district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    'Name of District/Department',
    'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]

def load_and_tag(path):
    """
    Read every sheet in the given Excel file, tag each with its sheet name as 'timeperiod',
    and return the list of DataFrames.
    """
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)
    return dfs

def load_all_excels_and_report(folder_path):
    """
    Load every .xls/.xlsx in folder_path, collect all sheets,
    and then print:
      - how many sheets per file have district columns
      - overall percentage of rows containing district info
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_dfs = []
    grand_with_sheets = 0
    grand_total_sheets = 0

    for path in sorted(excel_paths):
        # load sheets
        dfs = load_and_tag(path)

        # per-file sheet counts
        sheets_with = 0
        for df in dfs:
            if set(df.columns).intersection(district_columns):
                sheets_with += 1
        total_sheets = len(dfs)
        grand_with_sheets += sheets_with
        grand_total_sheets += total_sheets
        print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")

        all_dfs.extend(dfs)

    # concatenate everything
    big_df = pd.concat(all_dfs, ignore_index=True)

    # identify which of your district_columns actually made it in
    found_cols = list(set(big_df.columns).intersection(district_columns))
    if not found_cols:
        print("No district columns found at all!")
        return all_dfs

    # compute row‐level mask
    mask = big_df[found_cols].notna().any(axis=1)
    total_rows = len(big_df)
    rows_with_info = mask.sum()
    pct_with_info = rows_with_info / total_rows * 100

    print()
    print(f"Overall: {grand_with_sheets}/{grand_total_sheets} sheets contain district columns")
    print(f"Across all {total_rows:,} rows, {rows_with_info:,} ({pct_with_info:.2f}%) have district info")

    return all_dfs

# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_dataframes = load_all_excels_and_report(folder)


In [25]:
import os
import glob
import pandas as pd

def load_and_tag(path):
    """Read every sheet in the given Excel file, tag with its sheet name as timeperiod."""
    print(path)
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        print(sheet)
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)
    return dfs

def normalize_columns(df):
    """
    # Uncomment and customize this mapping if your sheets use different headers for the same field.
    col_map = {
        'Name of Distt.': 'district',
        'Name of District': 'district',
        'Sr. No.': 'sr_no',
        'Amount (In Crore)': 'amount_cr',
        # ... etc
    }
    return df.rename(columns=col_map)
    """
    return df  # no-op if you don't need normalization

def safe_outer_merge(dfs, on='timeperiod'):
    """If multiple DataFrames share the same timeperiod, outer-merge them,
       but error out if any overlapping non-key column has conflicting values."""
    from functools import reduce

    def merge_two(a, b):
        common = set(a.columns).intersection(b.columns) - {on}
        # find conflicts: for each row-key, column, both non-null and not equal
        merged = pd.merge(a, b, on=on, how='outer', suffixes=('_x','_y'))
        for col in common:
            x, y = col + '_x', col + '_y'
            # where both non-null and unequal
            conflict = merged[x].notna() & merged[y].notna() & (merged[x] != merged[y])
            if conflict.any():
                rows = merged[conflict][[on, x, y]]
                raise ValueError(f"Conflict in timeperiod={merged[on].iloc[0]}, column={col}:\n{rows}")
            # coalesce into single column
            merged[col] = merged[x].combine_first(merged[y])
            merged.drop([x, y], axis=1, inplace=True)
        return merged

    # group by timeperiod
    out = []
    for tp, group in pd.concat(dfs).groupby('timeperiod'):
        if tp == '':
            raise ValueError("Empty timeperiod found in input data")
                
        # if only one DataFrame for this timeperiod, just return it
        if len(group) == 1:
            out.append(group)
        else:
            # outer-merge all frames in this group
            frames = [g.drop(columns='timeperiod') for _, g in group.groupby(group.index)]
            merged = reduce(merge_two, [group[['timeperiod']].iloc[[0]].assign(**{})] + frames)
            out.append(merged)
    return pd.concat(out, ignore_index=True)

def build_master_table(folder):
    all_dfs = []
    for fname in glob.glob(os.path.join(folder, '*.xlsx')):
        all_dfs.extend(load_and_tag(fname))
    # normalize columns if needed
    all_dfs = [normalize_columns(df) for df in all_dfs]
    # try safe outer-merge for repeated timeperiods
    #master = safe_outer_merge(all_dfs)
    return pd.concat(all_dfs, ignore_index=True)

if __name__ == '__main__':
    folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
    master_df = build_master_table(folder)
    master_df
    #master_df.to_excel('master_table.xlsx', index=False)
    #print("Master table written to master_table.xlsx")
    


D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\Bank Sanction Order 2023-2024.xlsx
01-08-2023
23-08-2023
14-11-2023
23-01-2024
04-03-2024
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\CB Sanction Order 2023-2024.xlsx
16-09-2023
17-01-2024
31-01-2024
18-03-2024
30-03-2024
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndmf-sanction-order_2023-24.xlsx
10-04-2023
13-06-2023
11-07-2023
13-12-2023
09-01-2024
30-01-2024
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndrf-sanction-order_2022-23.xlsx
22-09-2022
22-09-2022_2
22-03-2023
22-03-2023_2
25-03-2023
29-03-2023
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndrf-sanct

In [26]:
master_df

,Name of Department,Account Head,Amount,timeperiod,Name of District/Department,Head of Account,Sr. No.,Name of Distt.,2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges),2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance),...,2245-02-101-01-S00N-NP(OC) Cash dole (Immediate relief),2245-02-106-01-S00N-NP(OC) Repairs & Restoration of Damaged roads & Bridges,"2245-02-109-01-S00N-NP(OC) R & R of Damaged Water Supply, Drainage & sewerage works",2245-02-193-01-S00N-NP(OC) Assistance to Local Bodies & other Non-Govt Bodies/Inst.,Amount (₹ cr),2245-02-111-01-S00N-NP(OC) Ex-Gratia Payments to Bereaved Families,2245-02-111-02-S00N-NP(OC) Ex-Gratia-II Death Due to State Specific Disaster,2245-02-111-02-S00N-NP(OC) Death Due to State Specific Disaster,2245-02-101-01-S00N-NP(OC) Cash Dole (Immediate Relief),2245-02-106-01-S00N-NP(OC) Repair & Restoration of Damaged Road & Bridges
0,HPPWD through Engineer-in-Chief HPPWD,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...,"Rs. 50,00,000/-",01-08-2023,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total,NaN,"​Rs. 50,00,000/-",01-08-2023,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"1,50,00,000/-",23-08-2023,Kangra,2245-02-113-01-S00N-NP(OC) Repair/Construction...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,"1,95,00,000/-",23-08-2023,Shimla,2245-02-113-01-S00N-NP(OC) Repair/Construction...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,"1,50,00,000/-",23-08-2023,Solan,2245-02-113-01-S00N-NP(OC) Repair/Construction...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,HPPWD through Engineer-in-Chief HPPWD (Divisio...,NaN,NaN,01-03-2024_2,NaN,2245-02-106-01-S00N-NP(OC) Repair & Restoratio...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,Jal Shakti Vibhag through Engineer-in-Chief Ja...,NaN,NaN,01-03-2024_2,NaN,2245-02-109-01-S00N-NP(OC) R & R of Damaged Wa...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778,NaN,NaN,NaN,01-03-2024_2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779,NaN,NaN,NaN,12-03-2024,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"50,00,000/-",NaN,NaN,NaN


In [27]:
master_df.columns.to_list()

['Name of Department',
 'Account Head',
 'Amount',
 'timeperiod',
 'Name of District/Department',
 'Head of Account',
 'Sr. No.',
 'Name of Distt.',
 '2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges)',
 '2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance)',
 '2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.)',
 'Total',
 'Name of District',
 'Budget Head',
 'Name of Work',
 'Amount (In Crore)',
 'Unnamed: 4',
 'Sub Minor Head',
 'Name of Building',
 'Total Amount Approved',
 'Amount of 1st Instalment (@30%)',
 'Total Amount Approved (₹)',
 'Already Released Amount (₹)',
 'Amount to be Released (₹)',
 'Unnamed: 7',
 'Unnamed: 8',
 'Unnamed: 9',
 'SN',
 'Hazard',
 'District',
 'Title of the Proposal',
 'Implementing Agencies',
 'Total Estimated Amount (In Rs.)',
 '1st Instalment Amount to be Released (₹) @29%',
 'Name of Districts',
 '05-2245-02-101-01-C00N-NP-(OC) Cash Dole (Immediate relief)',
 '05-2245-

In [33]:
import pandas as pd
import glob
import os

# Folder with your Excel files


excel_folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
excel_files = glob.glob(os.path.join(excel_folder, '*.xlsx'))
print(f"Found {len(excel_files)} Excel files in {excel_folder}")
# Initialize an empty list to hold DataFrames  
master_df = []
for file in excel_files:
    # Read the Excel file
    xls = pd.ExcelFile(file,engine='openpyxl')
    # Iterate through each sheet
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        print(df)
        # Add a new column with the sheet name
        df['timeperiod'] = sheet_name
        # Append the DataFrame to the list
        master_df.append(df)
master_df

Found 13 Excel files in D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables
                      Name of Department  \
0  HPPWD through Engineer-in-Chief HPPWD   
1                                  Total   

                                        Account Head            Amount  
0  2245-02-106-01-SOON-NP(OC) (Repair & Restorati...   Rs. 50,00,000/-  
1                                                NaN  ​Rs. 50,00,000/-  
  Name of District/Department  \
0                      Kangra   
1                      Shimla   
2                       Solan   
3                       Total   

                                     Head of Account         Amount  
0  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-  
1  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,95,00,000/-  
2  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-  
3                                                NaN  4,

[                      Name of Department  \
 0  HPPWD through Engineer-in-Chief HPPWD   
 1                                  Total   
 
                                         Account Head            Amount  \
 0  2245-02-106-01-SOON-NP(OC) (Repair & Restorati...   Rs. 50,00,000/-   
 1                                                NaN  ​Rs. 50,00,000/-   
 
    timeperiod  
 0  01-08-2023  
 1  01-08-2023  ,
   Name of District/Department  \
 0                      Kangra   
 1                      Shimla   
 2                       Solan   
 3                       Total   
 
                                      Head of Account         Amount  \
 0  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-   
 1  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,95,00,000/-   
 2  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-   
 3                                                NaN  4,95,00,000/-   
 
    timeperiod  
 0  23-08-2023  
 1  23-08-2023  

In [ ]:
master_df

[                      Name of Department  \
 0  HPPWD through Engineer-in-Chief HPPWD   
 1                                  Total   
 
                                         Account Head            Amount  \
 0  2245-02-106-01-SOON-NP(OC) (Repair & Restorati...   Rs. 50,00,000/-   
 1                                                NaN  ​Rs. 50,00,000/-   
 
    timeperiod  
 0  01-08-2023  
 1  01-08-2023  ,
   Name of District/Department  \
 0                      Kangra   
 1                      Shimla   
 2                       Solan   
 3                       Total   
 
                                      Head of Account         Amount  \
 0  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-   
 1  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,95,00,000/-   
 2  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-   
 3                                                NaN  4,95,00,000/-   
 
    timeperiod  
 0  23-08-2023  
 1  23-08-2023  